In [108]:
import numpy as np
import pandas as pd

# Take input from the user
## Nodes= {Node Number:[Volume(m3), Type]}

## Type: 
      Book Store=b
      Fish Shop=f
      Restaurant=r
      Garage=g
      Construction Company=c
      Lab=l
      Vegetables=v
      Electronics=e

In [109]:
#cap=int(input("Please enter the capacity of the garabage truck: "))
# n=int(input("Please enter the number of nodes: "))

In [110]:
# nodes={}
# for i in range (n):
#     V=int(input("Please enter the volume (m3) of the container " + str(i+1)+": "))
#     T=input("Please enter the type of the facility " + str(i+1)+": ")
#     deets=[V,T]
#     nodes[i+1]=deets

# nodes

# If input is sample

In [111]:
#Assuming vehicle capacity
vcap=100
nodes={1: [20, 'b'], 2: [15, 'b'], 3: [20, 'f'], 4: [18, 'r'], 5: [15, 'g'], 6: [20, 'c'], 7: [20, 'b'], 8: [18, 'l'], 9: [20, 'v'], 10: [20, 'c'], 11: [18, 'f'], 12: [20, 'e'], 13: [20, 'v'], 14: [15, 'r'], 15: [20, 'g'], 16: [20, 'b'], 17: [20, 'b'], 18: [20, 'f'], 19: [18, 'r'], 20: [15, 'g'], 21: [20, 'c'], 22: [20, 'b'], 23: [18, 'l'], 24: [20, 'v'], 25: [20, 'c'], 26: [18, 'r'], 27: [20, 'e'], 28: [20, 'v'], 29: [20, 'r'], 30: [20, 'g']}
nc=12
eoc=24
wpf_dan=[1]
wpf_gen=[4,6,15]
aht=[1,13,19,24]

In [112]:
#Read and prepare the nodes distance matrix
ndm = pd.read_csv('input/nodes_dm.csv')
ndm=ndm.astype('int')
ndm=ndm.set_index('ID')
ndm=ndm.sort_values(by='ID')
ndm.columns=ndm.columns.astype('int')
fdm=ndm.sort_index(axis=1)

In [113]:
#Read and prepare the depot distance matrix
ddm = pd.read_csv('input/depot_dm.csv')
ddm=ddm.astype('int')
ddm=ddm.drop('InputID', axis=1)
ddm=ddm.set_index('TargetID')
ddm=ddm.sort_values(by='TargetID')
# ndm.columns=ndm.columns.astype('int')
# ndm=ndm.sort_index(axis=1)

# Create a function that applies Savings heuristic (Clarke-Wright) on a list of nodes

In [114]:
#define a function that creates a dataframe of savings for all pairs of nodes in a tuple
def savi(tup):
    savings=dict()
    pairs= [(a, b) for idx, a in enumerate(tup) for b in tup[idx + 1:]]
    for i in pairs:
        savings[i]=ddm['Distance'][i[0]] + ddm['Distance'][i[1]] - fdm[i[0]][i[1]]
    # put savings in a pandas dataframe, and sort by descending
    sv = pd.DataFrame.from_dict(savings, orient = 'index')
    sv.rename(columns = {0:'saving'}, inplace = True)
    sv.sort_values(by = ['saving'], ascending = False, inplace = True)
    return sv

pd.set_option('display.max_rows', None)

In [115]:
#check how many nodes from a saving are there in the existing list of routes
def checksav(lor, sav):
    node1=any(sav[0] in sublist for sublist in lor)
    node2=any(sav[1] in sublist for sublist in lor)
    if node1 is False and node2 is False:
        return(1)
    elif node1 is True and node2 is True:
        return(2)
    else:
        if node1 == True:
            return(3)
        else:
            return(4)

In [116]:
#given a node is there in the lor, get the index of the route in the lor and the index of that node in the route
def che_ind(lor, n):
    for i in lor:
        if n in i:
            return(lor.index(i), i.index(n))
            break
        else:
            continue

In [117]:
# merge two routes with a connection link
def merge(r1, r2, li):
    if r1[-1] in li and r2[0] in li:
        return(r1+r2)
    elif r1[0] in li and r2[0] in li:
        r1.reverse()
    elif r1[-1] in li and r2[-1] in li:
        r2.reverse()
    else:
        r1.reverse()
        r2.reverse()
    return(r1+r2)

In [118]:
#define a function to find the total demand of a route
def r_dem(x):
    ds=0
    for i in x:
        ds=ds+nodes[i][0]
    return(ds)

# Pick the biggest saving:

### 1. No exisitng route has any of the nodes. Create a new route. Add the saving to new route. CONTINUE.
    The problem with this is that it will create routes with demands way less than the vehicel capacity.

### 2. Both nodes are there in the existing routes:
    2.1. Both nodes are in the same route. CONTINUE.
    2.2. Both nodes are in the different routes:
        2.2.1. Both nodes are NOT in the interior. Merge both routes. CONTINUE.        
        2.2.2. Atleast one node is in the interior. CONTINUE.
            
### 3. One exisitng route has one of the nodes:
    3.1. The node is in the interior of the exisitng route. CONTINUE.
    3.2. The node is NOT in the interior of the existing route:
        3.2.1. Node is at the beginning of the route. Add the other node at the beginning of that route. CONTINUE.
        3.2.2. Node is at the end of the route. Add the other node at the end of that route. CONTINUE.

In [119]:
#define a function that performs savings heuristic on a list of nodes for a certain vehicle capacity
def cwalgo(lis):
    
    #list of routes
    lor=[]
    cap=vcap

    #create savings dataframe
    sv=savi(lis)

    #run a for loop through all the savings
    for sav in sv.index:
        ##1. No node in the exisitng list of routes(lor)
        if checksav(lor, sav)==1:
            #check if the demand of the 2 nodes exceeds the capacity
            if nodes[sav[0]][0]+nodes[sav[1]][0]<cap:
                #create a new route
                r=[]
                r.append(sav[0])
                r.append(sav[1])

                #add the new route in the list of routes
                lor.append(r)
            else:
                continue
            
        ##2. Both nodes already in the existing list of routes(lor)
        elif checksav(lor, sav)==2:
            ##2.1 both nodes in the same route
            if che_ind(lor,sav[0])[0]==che_ind(lor,sav[1])[0]:
                continue
            ##2.2 nodes are in different routes
            else:
                ##2.2.1 Check if the BOTH nodes are the end nodes (NOT in the interior) of the respective route
                i=che_ind(lor,sav[0])[1]
                routei=lor[che_ind(lor,sav[0])[0]]
                j=che_ind(lor,sav[1])[1]
                routej=lor[che_ind(lor,sav[1])[0]]
                if (i==0 or i==len(routei)-1) and (j==0 or j==len(routej)-1):
                    #merge routes (if the capacity allows)
                    route1=lor[che_ind(lor,sav[0])[0]]
                    route2=lor[che_ind(lor,sav[1])[0]]
                    if r_dem(route1) + r_dem(route2)<=cap:
                        lor.remove(route1)
                        lor.remove(route2)
                        lor.append(merge(route1,route2,sav))
                    else:
                        continue
                else:
                    continue
                    
        ##3. If the 1st node is there in the already exising list of routes            
        elif checksav(lor, sav)==3:
            #index of node in the route
            route=lor[che_ind(lor,sav[0])[0]]
            node_ind=che_ind(lor,sav[0])[1]
            if r_dem(route)+nodes[sav[1]][0]>cap:
                continue
            else:
                ##3.1 if the node is in the interior of the route
                if node_ind!=0 and node_ind!=len(route)-1:
                    continue
                ##3.2 Node is at the exterior of the route
                ###3.2.1 Node is at the beginning of the route
                elif node_ind==0:
                    route.insert(0,sav[1])
                else:
                    route.insert(len(route),sav[1])
                
        ##3. If the 2nd node is there in the already exising list of routes
        else:
            #index of node in the route
            route=lor[che_ind(lor,sav[1])[0]]
            node_ind=che_ind(lor,sav[1])[1]
            if r_dem(route)+nodes[sav[0]][0]>cap:
                continue
            else:
                ##3.1 if the node is in the interior of the route
                if node_ind!=0 and node_ind!=len(route)-1:
                    continue
                        
                ##3.2 Node is at the exterior of the route
                ###3.2.1 Node is at the beginning of the route
                elif node_ind==0:
                    route.insert(0,sav[0])
                ###3.2.2 Node is at the end of the route
                else:
                    route.insert(len(route),sav[0])
    return lor

In [120]:
#define a function to calculate the total distance travelled in a route from depot to last node
def tdr(route):
    td=ddm['Distance'][route[0]]
    for x in range(len(route)-1):
        i=route[x]
        j=route[x+1]
        td=td+fdm[i][j]
    return td

In [121]:
#define a function to calculate the total distance travelled in a list of routes
def tdlor(lor):
    td=0
    for i in lor:
          td=td+tdr(i)
    return (td)

In [122]:
#bifurcate dangerous, construction, general waste nodes
dan=[]
con=[]
gen=[]
for i in nodes:
    if i==nc:
        con.append(i)
    elif nodes[i][1]=='g' or nodes[i][1]=='l' or nodes[i][1]=='e':
        dan.append(i)
    elif nodes[i][1]=='c':
        con.append(i)
    else:
        gen.append(i)

### Put every construction waste node in dangerous and general waste groups to check which gives the minimum increment in distance

In [123]:
#check location for every construction waste node
for i in con:
    #distance of original dangerous list of routes
    dlor = cwalgo(dan)
    tddlor = tdlor(dlor)
    
    #distance of original dangerous list of routes
    glor = cwalgo(gen)
    tdglor = tdlor(glor)
    
    #put in dangerous waste nodes
    dan.append(i)
    #calculate new total distance
    dlor_new = cwalgo(dan)
    tddlor_new = tdlor(dlor_new)
    #difference in distances
    diff_d = tddlor_new-tddlor
    
    #put in general waste nodes
    gen.append(i)
    #calculate new total distance
    glor_new = cwalgo(gen)
    tdglor_new = tdlor(glor_new)
    #difference in distances
    diff_g = tdglor_new-tdglor
    
    if diff_g > diff_d:
        gen.remove(i)
    else:
        dan.remove(i)

In [124]:
gen_lor=cwalgo(gen)
gen_lor

[[1, 6, 19, 29, 28],
 [24, 9, 16, 17, 18],
 [26, 12, 11, 4],
 [14, 22, 3, 10, 7],
 [2, 13]]

In [125]:
dan_lor=cwalgo(dan)
dan_lor

[[5, 20, 15, 21], [30, 25, 8, 27, 23]]

## Insert the wpf nodes at the end of every route

In [126]:
#make separate list of routes serving a wpf node and those that are not serving a wpf node
wpfr=[]
nwpfr=[]


In [127]:
#dangerous waste routes

for r in dan_lor:
    
    #if there is a wpf node in the route, put the wpf node in the end
    if any(n in r for n in wpf_dan)==True:
        inter=[i for i in r if i in wpf_dan][0]
        r.remove(inter)
        r.append(inter)
        wpfr.append(r)
        
    #if there is no wpf node in a route, add the wpf node at the end
    else:
        r.append(wpf_dan[0])
        nwpfr.append(r)
    
dan_lor

[[5, 20, 15, 21, 1], [30, 25, 8, 27, 23, 1]]

In [128]:
#general waste routes

for r in gen_lor:
    
    #if there is a wpf node in the route, put the wpf node in the end
    if any(n in r for n in wpf_gen)==True:
        inter=[i for i in r if i in wpf_gen][0]
        r.remove(inter)
        r.append(inter)
        wpfr.append(r)
    
    #when there is no wpf node in any route, add a wpf node which is closest to the last node of the route
    else:
        lnd=[]
        for j in wpf_gen:
            lndj=fdm[r[-1]][j]
            lnd.append(lndj)
        minlnd=min(lnd)
        mind=lnd.index(minlnd)
        r.append(wpf_gen[mind])
        nwpfr.append(r)
            
gen_lor

[[1, 19, 29, 28, 6],
 [24, 9, 16, 17, 18, 4],
 [26, 12, 11, 4],
 [14, 22, 3, 10, 7, 4],
 [2, 13, 4]]

In [129]:
#all routes
allr=wpfr+nwpfr
wpfr,nwpfr

([[1, 19, 29, 28, 6], [26, 12, 11, 4]],
 [[5, 20, 15, 21, 1],
  [30, 25, 8, 27, 23, 1],
  [24, 9, 16, 17, 18, 4],
  [14, 22, 3, 10, 7, 4],
  [2, 13, 4]])

# Time Constraints
### Normal time window (Departure of trucks from the depot to arrival of trucks at the depot)= 7 am to 5 pm (0-600)
### Every customer should be served before 4 pm (< 540)
### All the restraunts should be served before noon (< 300)
### Dangerous waste can only be dumped between 1 pm and 4 pm (360-540)

# Time insertions
### Every truck takes 10 minutes to dump the waste at the facility (assumption)
### Every node except bookstores need 10 minutes (assumption) for service, book stores need 15 minutes (assumption) for service as they need the same container back
### Additional Handeling time of 15 minutes at [aht] nodes

In [130]:
#list of book stores and restaurants
book=[]
rest=[]
for i in nodes:
    if nodes[i][1]=='b':
        book.append(i)
    elif nodes[i][1]=='r':
        rest.append(i)
    else:
        continue

In [131]:
#Read and prepare the nodes time matrix
ntm = pd.read_csv('input/nodes_tm.csv')
ntm=ntm.astype('int')
ntm=ntm.set_index('ID')
ntm=ntm.sort_values(by='ID')
ntm.columns=ntm.columns.astype('int')
ntm=ntm.sort_index(axis=1)

In [132]:
#Read and prepare the depot time matrix
dtm = pd.read_csv('input/depot_tm.csv')
dtm=dtm.astype('int')
dtm=dtm.drop('InputID', axis=1)
dtm=dtm.set_index('TargetID')
dtm=dtm.sort_values(by='TargetID')

In [133]:
#define a function that give times for every node of a route at a given starting point
def tw(route, start):
    t={}
    t['dep1']=start
    time=start + dtm['Time'][route[0]]
    
    #run a loop through all the nodes except the last node (wpf node)
    for i in range(len(route)-1):
        
        #Add Service time at node i
        if route[i] in book and route[i] in aht:
            time = time + 30
        elif route[i] in aht:
            time = time + 25
        elif route[i] in book:
            time = time + 15
        else:
            time = time + 10
        
        #Add into the 't' dictionary
        t[route[i]] =time
        
        #Add the travel time from (i) to (i+1)
        time = time + ntm[route[i]][route[i+1]]
        
    #Add service time at the wpf node
    
    w=route[-1]
    
    #if route has to serve as well dump at wpf node
    if route in wpfr:
              
        #service time
        if w in book and w in aht:
            time = time + 30
        elif w in aht:
            time = time + 25
        elif w in book:
            time = time + 15
        else:
            time = time + 10
            
        #Add w processing time into the 't' dictionary
        t[w] = time
        
        #dumping time
        time = time + 10
        
    else:
        #only dumping time
        time = time + 10
    
    #Add dumping completion into the 't' dictionary
    t['dump'] = time
    
    #Add returning to the depot time
    time=time+dtm["Time"][w]
    t['dep2']=time
    
    return(t)

In [134]:
#combine the service completion times of all the nodes in a dictinary
strt=0
nsct={}
dumpct=[]
deprt=[]
for i in allr:
    dic=tw(i,strt)
    for j in dic:
        if type(j)==int:
            nsct[j]=dic[j]
        elif j=='dump':
            dumpct.append((i[-1], dic[j]))
        elif j=='dep2':
            deprt.append((i[-1], dic[j]))

## Create a dictionary with keys as starting times and values as routes

In [135]:
gen_lor

[[1, 19, 29, 28, 6],
 [24, 9, 16, 17, 18, 4],
 [26, 12, 11, 4],
 [14, 22, 3, 10, 7, 4],
 [2, 13, 4]]

In [136]:
final_d=[]

In [137]:
#ensure restraunts are served before noon
for i in gen_lor:
    
    #check if there is a restaurant node in the route (before last node)
    dupi=i
    for j in range(len(dupi)-1):
        if dupi[j] in rest and nsct[dupi[j]]>300:
            n=dupi[j]
            while (tw(i,0))[n]>300:
                nind=i.index(n)
                i.remove(n)
                i.insert(nind-1)
            
    #for last node
    if i in wpfr and i[-1] in rest and nsct[i[-1]]>300:
        n=dupi[-1]
        nind=i.index(n)
        i.insert(nind-1)
        while (tw(i,0))[n]>300:
                nind=i.index(n)
                i.remove(n)
                i.insert(nind-1)
                
for i in gen_lor:
    final_d.append((0,i))

In [138]:
#ensure dangerous waste is dumped in between 1 pm and 4 pm
for i in dan_lor:
    strt=0
    times=tw(i,strt)
    if times['dump']<370:
        diff=370-times['dump']
        final_d.append((diff,i))
    else:
        final_d.append(0,i)

In [139]:
final_d

[(0, [1, 19, 29, 28, 6]),
 (0, [24, 9, 16, 17, 18, 4]),
 (0, [26, 12, 11, 4]),
 (0, [14, 22, 3, 10, 7, 4]),
 (0, [2, 13, 4]),
 (131, [5, 20, 15, 21, 1]),
 (120, [30, 25, 8, 27, 23, 1])]

In [140]:
for i in range(len(final_d)):
    start=(final_d[i])[0]
    route=(final_d[i])[1]
    print("Truck: "+str(i+1))
    print(tw(route,start))

Truck: 1
{'dep1': 0, 1: 63, 19: 211, 29: 250, 28: 287, 6: 338, 'dump': 348, 'dep2': 407}
Truck: 2
{'dep1': 0, 24: 61, 9: 124, 16: 180, 17: 223, 18: 253, 'dump': 293, 'dep2': 304}
Truck: 3
{'dep1': 0, 26: 31, 12: 47, 11: 67, 4: 87, 'dump': 97, 'dep2': 108}
Truck: 4
{'dep1': 0, 14: 23, 22: 38, 3: 48, 10: 68, 7: 104, 'dump': 150, 'dep2': 161}
Truck: 5
{'dep1': 0, 2: 28, 13: 63, 'dump': 95, 'dep2': 106}
Truck: 6
{'dep1': 131, 5: 150, 20: 169, 15: 268, 21: 319, 'dump': 370, 'dep2': 403}
Truck: 7
{'dep1': 120, 30: 161, 25: 202, 8: 243, 27: 288, 23: 320, 'dump': 370, 'dep2': 403}
